In [1]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster

import json
import pandas as pd
import branca

In [2]:
from utils_data import get_contours_city_ign, merge_iris_pop_and_traffic, get_adjacency_from_gdf 

In [3]:
crs_angles='EPSG:4326'
crs_meters='EPSG:2154'
waiting_time_api=0.1
code_insee = 44055

In [8]:
file_ign_region = '../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'
file_iris_pop_demand = f"../data/processed/demand_per_iris_hab_{code_insee}.geojson"
file_iris_traffic = f"../data/processed/traffic_demand_per_iris_{code_insee}.geojson"

In [9]:
export_file_iris_all_demand = f"../data/processed/all_demand_per_iris_{code_insee}.geojson"
export_file_df_adjacency=f"../data/processed/gdf_city_iris_with_dist_{code_insee}.csv"
export_file_np_adjacency=f"../data/processed/adjacency_matrix_{code_insee}.npy"

# Adding both demands

In [10]:
gdf_iris_traffic = gpd.read_file(file_iris_traffic)

In [11]:
gdf_iris_traffic

,code_iris,nom_iris,demand_traffic_kWh,geometry
0,440550101,La Baule les Pins,0.000000,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47..."
1,440550102,Centre-Benoît,0.000000,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47..."
2,440550103,Gare-Grand Clos,0.000000,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47...."
3,440550104,Beslon,0.000000,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47..."
4,440550105,Escoublac,52.770902,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47..."
5,440550106,Le Guézy,70.929473,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3..."


In [16]:
gdf_iris_traffic.demand_traffic_kWh.sum()

123.70037530292733

In [13]:
gdf_iris_population = gpd.read_file(file_iris_pop_demand)

In [14]:
gdf_iris_population

,code_iris,nom_iris,revenues,population,density_km2,demand_pop_kWh,geometry
0,440550101,La Baule les Pins,31760,2346.994576,1290.289305,463.191521,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47..."
1,440550102,Centre-Benoît,31940,1616.919056,1029.639154,321.254665,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47..."
2,440550103,Gare-Grand Clos,26750,1869.032071,1584.310324,317.430572,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47...."
3,440550104,Beslon,23160,2408.488869,1057.621831,206.515939,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47..."
4,440550105,Escoublac,27110,3511.317978,421.501327,218.825894,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47..."
5,440550106,Le Guézy,27800,4467.253898,394.213733,282.994680,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3..."


In [15]:
gdf_iris_population.demand_pop_kWh.sum()

1810.2132709118382

In [10]:
gdf_iris_all_demands = merge_iris_pop_and_traffic(gdf_iris_population, gdf_iris_traffic)
# gdf_iris_all_demands.head()

In [11]:
gdf_iris_all_demands

,code_iris,nom_iris,demand_pop_kWh,geometry,demand_traffic_kWh,total_demand_kWh
0,440550101,La Baule les Pins,463.191521,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",0.000000,463.191521
1,440550102,Centre-Benoît,321.254665,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",0.000000,321.254665
2,440550103,Gare-Grand Clos,317.430572,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",0.000000,317.430572
3,440550104,Beslon,206.515939,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",0.000000,206.515939
4,440550105,Escoublac,218.825894,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",52.770902,271.596796
5,440550106,Le Guézy,282.994680,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3...",70.929473,353.924153


# Plot

In [12]:
gdf_ign_city, geometry_ign_city = get_contours_city_ign(file_ign_region, code_insee)
center_lat, center_lon = geometry_ign_city.centroid.y, geometry_ign_city.centroid.x

In [13]:
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Create a colormap based on the number of people in each carreau
min_demand = gdf_iris_all_demands['total_demand_kWh'].min()  # Minimum number of people
max_demand = gdf_iris_all_demands['total_demand_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_demand, max_demand)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_iris_all_demands.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['total_demand_kWh']  # Assuming this column exists
    nom = row['nom_iris']  # Assuming this column exists

    # Create a popup displaying the number of people
    text = f"{nom} : Total demand={demand:.2f}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)
    
# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

# Get distance for the model

In [14]:
gdf_city_iris_with_dist, adjacency_matrix = get_adjacency_from_gdf(gdf_iris_all_demands, delay=waiting_time_api, crs_angles=crs_angles, crs_meters=crs_meters)

In [15]:
gdf_city_iris_with_dist.head()

,code_iris,nom_iris,demand_pop_kWh,geometry,demand_traffic_kWh,total_demand_kWh,centroid,longitude,latitude,dist_0,dist_1,dist_2,dist_3,dist_4,dist_5
0,440550101,La Baule les Pins,463.191521,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",0.000000,463.191521,POINT (-2.36782 47.28052),-2.367825,47.280523,0.0000,2.8856,3.3062,2.3819,2.9707,4.3403
1,440550102,Centre-Benoît,321.254665,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",0.000000,321.254665,POINT (-2.40124 47.2817),-2.401235,47.281697,2.8856,0.0000,0.7609,3.3710,7.4203,11.3524
2,440550103,Gare-Grand Clos,317.430572,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",0.000000,317.430572,POINT (-2.4036 47.28485),-2.403605,47.284850,3.3062,0.7609,0.0000,3.0541,7.0775,11.0097
3,440550104,Beslon,206.515939,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",0.000000,206.515939,POINT (-2.38227 47.29256),-2.382273,47.292557,2.3819,3.3710,3.0541,0.0000,2.6009,5.0269
4,440550105,Escoublac,218.825894,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",52.770902,271.596796,POINT (-2.35904 47.30102),-2.359042,47.301024,2.9707,7.4203,7.0775,2.6009,0.0000,4.1328


In [19]:
cols = ['nom_iris', 'geometry', 'demand_pop_kWh', 'demand_traffic_kWh', 'total_demand_kWh']
cols += [f'dist_{i}' for i in range(len(gdf_city_iris_with_dist))]
gdf_city_iris_with_dist.columns

Index(['code_iris', 'nom_iris', 'demand_pop_kWh', 'geometry',
       'demand_traffic_kWh', 'total_demand_kWh', 'centroid', 'longitude',
       'latitude', 'dist_0', 'dist_1', 'dist_2', 'dist_3', 'dist_4', 'dist_5'],
      dtype='object')

In [20]:
gdf_city_iris_with_dist_clean = gdf_city_iris_with_dist[cols]
gdf_city_iris_with_dist_clean

,nom_iris,geometry,demand_pop_kWh,demand_traffic_kWh,total_demand_kWh,dist_0,dist_1,dist_2,dist_3,dist_4,dist_5
0,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",463.191521,0.000000,463.191521,0.0000,2.8856,3.3062,2.3819,2.9707,4.3403
1,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",321.254665,0.000000,321.254665,2.8856,0.0000,0.7609,3.3710,7.4203,11.3524
2,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",317.430572,0.000000,317.430572,3.3062,0.7609,0.0000,3.0541,7.0775,11.0097
3,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",206.515939,0.000000,206.515939,2.3819,3.3710,3.0541,0.0000,2.6009,5.0269
4,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",218.825894,52.770902,271.596796,2.9707,7.4203,7.0775,2.6009,0.0000,4.1328
5,Le Guézy,"MULTIPOLYGON (((-2.29702 47.3032, -2.2972 47.3...",282.994680,70.929473,353.924153,4.3403,11.3524,11.0097,5.0269,4.1328,0.0000


In [21]:
gdf_city_iris_with_dist.to_csv(export_file_df_adjacency, index=False)
np.save(export_file_np_adjacency, adjacency_matrix)

# Loading from saved file

In [24]:
gdf_iris_all_demands = gpd.read_file(export_file_df_adjacency)
gdf_iris_all_demands

,nom_iris,geometry,demand_pop_kWh,demand_traffic_kWh,total_demand_kWh,dist_0,dist_1,dist_2,dist_3,dist_4,dist_5
0,La Baule les Pins,MULTIPOLYGON (((-2.351325998650614 47.27243657...,463.19152100444137,0.0,463.19152100444137,0.0,2.8855999999999997,3.3062,2.3819,2.9707,4.3403
1,Centre-Benoît,MULTIPOLYGON (((-2.381270239151754 47.27992660...,321.2546653491017,0.0,321.2546653491017,2.8855999999999997,0.0,0.7609,3.371,7.4203,11.3524
2,Gare-Grand Clos,MULTIPOLYGON (((-2.4272956419199745 47.2783327...,317.4305722042825,0.0,317.4305722042825,3.3062,0.7609,0.0,3.0541,7.0775,11.0097
3,Beslon,MULTIPOLYGON (((-2.404911812815671 47.28722700...,206.51593906066302,0.0,206.51593906066302,2.3819,3.371,3.0541,0.0,2.6009,5.0268999999999995
4,Escoublac,MULTIPOLYGON (((-2.3897656047026707 47.3000943...,218.82589374196306,52.77090205021432,271.5967957921774,2.9707,7.4203,7.0775,2.6009,0.0,4.1328000000000005
5,Le Guézy,MULTIPOLYGON (((-2.297022884553247 47.30319607...,282.9946795513867,70.92947325271301,353.9241528040997,4.3403,11.3524,11.0097,5.0268999999999995,4.1328000000000005,0.0
